In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt_train = date(2013, 1, 1)
end_dt_train = date(2017, 8, 15)
test_dt = date(2016, 5,10)
train_dates1 = []
dataTypes = []
for dt in daterange(start_dt_train, end_dt_train):
    train_dates1.append(dt.strftime("%Y-%m-%d"))
    if dt > test_dt:
        dataTypes.append('test')
    else:
        dataTypes.append('train')
    
start_dt_test = date(2017, 8, 16)
end_dt_test = date(2017, 8, 31)
test_dates = []
for dt in daterange(start_dt_test, end_dt_test):
    test_dates.append(dt.strftime("%Y-%m-%d"))
    dataTypes.append('val')
all_dates = []
for dt in daterange(start_dt_train, end_dt_test):
    all_dates.append(dt.strftime("%Y-%m-%d"))

In [ ]:
datesDf = pd.DataFrame(data=all_dates, columns=['date'])

In [ ]:
holidays = pd.read_csv("holidays_events.csv")
oil = pd.read_csv("oil.csv")
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv")

In [ ]:
holidays = holidays.merge(datesDf, how='inner')
oil = oil.merge(datesDf, how = 'inner')

In [ ]:
# add weekday, month, year
day = []
month = []
year =[]
weekday =[]
for dt in daterange(start_dt_train, end_dt_test):
    day.append(dt.day)
    weekday.append(dt.weekday())
    month.append(dt.month)
    year.append(dt.year)

datesDf = datesDf.assign(weekday = weekday)
datesDf = datesDf.assign(day = day)
datesDf = datesDf.assign(month = month)
datesDf = datesDf.assign(year = year)
datesDf = datesDf.assign(dataType = dataTypes)
datesDf

In [ ]:
# add oil that is the same for every store
datesDf = datesDf.merge(oil, how='outer')
datesDf.dcoilwtico.interpolate(method='linear', inplace=True) #probably don't need to normalize since in good range
datesDf['dcoilwtico'] = datesDf.dcoilwtico.fillna(93.140000)
datesDf

In [ ]:
#add holiday information
datesDf = datesDf.merge(holidays, how = 'outer')
datesDf

In [ ]:
#encode stuff
print(datesDf.type.value_counts())

print(datesDf.locale.value_counts())

print(datesDf.locale_name.value_counts()) #attention! some local names match store locations

print(datesDf.description.value_counts(), datesDf.description.unique())

In [ ]:
stores

In [ ]:
set(stores.state.value_counts()) & set(datesDf.locale_name.value_counts())

In [ ]:
set(stores.city.value_counts()) & set(datesDf.locale_name.value_counts())

In [ ]:
storeColumns = stores.columns

In [ ]:
storeIds = np.sort(transactions.store_nbr.unique())

df = pd.DataFrame()
for storeId in storeIds:
    #add transaction info
    df_loc = datesDf.merge(transactions.loc[transactions.store_nbr==storeId], how='outer')
    df_loc.store_nbr.fillna(storeId, inplace=True)
    df_loc.transactions.fillna(0, inplace=True)
    
    # add store info 
    df_loc = df_loc.assign(city = stores.loc[(stores.store_nbr == storeId, 'city')].to_numpy()[0])
    df_loc = df_loc.assign(state = stores.loc[(stores.store_nbr == storeId, 'state')].to_numpy()[0])
    df_loc = df_loc.assign(citytype = stores.loc[(stores.store_nbr == storeId, 'type')].to_numpy()[0])
    df_loc = df_loc.assign(cluster = stores.loc[(stores.store_nbr == storeId, 'cluster')].to_numpy()[0])

    #check if store location is same as holiday location -> create local holiday column
    df_loc['holidayAtStoreState'] = df_loc['state'] == df_loc['locale_name']
    df_loc['holidayAtStoreCity'] = df_loc['city'] == df_loc['locale_name']
    
    # add id to train data (time relation)
    df_loc['index'] = df_loc.index
    
    df = pd.concat([df,df_loc])
    
    #fig = plt.figure()
    #df_loc.transactions.plot()
    
df

In [ ]:
le = LabelEncoder()
cols = ['type','locale','locale_name','description','transferred', 'city','state','citytype', 'holidayAtStoreState','holidayAtStoreCity']
#
# Encode labels of multiple columns at once
#
df[cols] = df[cols].apply(LabelEncoder().fit_transform)
df

In [ ]:
df.to_csv('timeDF.csv')